In [1]:
import pandas as pd

In [34]:
train = pd.read_csv('../data/train_nc_nctools-predictions.txt', sep='\t')
train.shape

(81140, 158)

In [6]:
#remm = pd.read_csv('../data/nctools_predictions/ReMM_predictions.txt', sep='\s')
ncboost = pd.read_csv('../data/nctools_predictions/NCBoost_predictions.txt', sep='\t')
#deepsea = pd.read_csv('../files/nctools_predictions/deepsea_jobs/infile.vcf.out.funsig', index_col=0)
#eigen = pd.read_csv('../data/nctools_predictions/eigen_filtered.txt', sep='\t')
#remm.shape, ncboost.shape, deepsea.shape, eigen.shape

Add the predictions of the nctools to the train set. For ReMM and NCBoost, the predictions are position based for SNV's. Therefore, they can only be merged with SNV variants from the training set. 

<h1>Add ReMM predictions to train set</h1>

As the ReMM scores are comupted only for SNVs and don't contain the alleles in the file, first merge the ReMM df with a subset of the train set only containing SNV's. Then merge with the total train set. 

In [147]:
train_snvs = train[train.Ref.apply(lambda x: len(x) == 1) & train.Alt.apply(lambda x: len(x) == 1)]
print(train_snvs.shape)

(70440, 152)


In [148]:
train_snvs['#Chrom'] = train_snvs['#Chrom'].apply(lambda x: str(x)) #convert all chromosmes to string (some are int)
train_snvs = train_snvs[['#Chrom', 'Pos', 'Ref', 'Alt']] 

//anaconda3/envs/py3_vp/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [149]:
train_snvs

,#Chrom,Pos,Ref,Alt
52,13,31891702,G,A
53,16,3788684,G,C
54,20,10621760,C,T
55,20,10653347,A,G
56,20,10629706,T,C
...,...,...,...,...
81135,1,220160782,T,A
81136,17,4805502,C,T
81137,2,179442915,A,G
81138,22,33253960,C,G


In [150]:
remm.columns = ['#Chrom', 'Pos', 'ReMM_score']
remm

,#Chrom,Pos,ReMM_score
0,1,874714,0.905
1,1,874816,0.253
2,1,899926,0.490
3,1,905775,0.995
4,1,978847,0.936
...,...,...,...
76765,X,154489082,0.197
76766,X,154489806,0.798
76767,X,154507173,0.411
76768,X,154563670,0.843


Merge the ReMM df with the train SNV's to get the correct reference and alternative alleles for the ReMM scores.

In [151]:
remm_train = train_snvs.merge(remm, on=['#Chrom', 'Pos'], how='inner')

In [152]:
remm_train

,#Chrom,Pos,Ref,Alt,ReMM_score
0,13,31891702,G,A,0.964
1,16,3788684,G,C,0.005
2,20,10621760,C,T,0.998
3,20,10621760,C,A,0.998
4,20,10653347,A,G,0.997
...,...,...,...,...,...
70435,1,220160782,T,A,0.644
70436,17,4805502,C,T,0.085
70437,2,179442915,A,G,0.069
70438,22,33253960,C,G,0.027


Finally, add them to the total train set. 

In [35]:
train.shape


(81140, 158)

In [153]:
train = train.merge(remm_train, on=['#Chrom', 'Pos', 'Ref', 'Alt'], how='left')

In [154]:
train.shape

(81140, 153)

<h1>Add NCBoost predictions to train set</h1>

TODO: use NCBoost to compute predictions for the variants in the train set. The precomputed scores only contain a subset of variants.

In [36]:
train_snvs = train[train.Ref.apply(lambda x: len(x) == 1) & train.Alt.apply(lambda x: len(x) == 1)]
train_snvs['#Chrom'] = train_snvs['#Chrom'].apply(lambda x: str(x)) #convert all chromosmes to string (some are int)
train_snvs = train_snvs[['#Chrom', 'Pos', 'Ref', 'Alt']] 
print(train_snvs.shape)

(70440, 4)


//anaconda3/envs/py3_vp/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [37]:
ncboost.columns = ['#Chrom', 'Pos', 'NCBoost_region', 'NCBoost_closest_gene_name', 'NCBoost_Score', 'NCBoost_chr_rank_perc']
ncboost

,#Chrom,Pos,NCBoost_region,NCBoost_closest_gene_name,NCBoost_Score,NCBoost_chr_rank_perc
0,1,899926,intronic,KLHL17,0.095736,0.758251
1,1,980669,intronic,AGRN,0.164826,0.866997
2,1,980948,intronic,AGRN,0.111841,0.792821
3,1,981087,intronic,AGRN,0.131175,0.825299
4,1,981328,intronic,AGRN,0.255828,0.930296
...,...,...,...,...,...,...
36116,X,154489048,UTR3,RAB39B,0.210673,0.689868
36117,X,154489082,UTR3,RAB39B,0.192254,0.669302
36118,X,154489806,UTR3,RAB39B,0.345932,0.822775
36119,X,154507173,UTR3,CLIC2,0.219785,0.699784


In [38]:
ncboost_train = train_snvs.merge(ncboost, on=['#Chrom', 'Pos'], how='inner')

In [39]:
ncboost_train.shape

(31654, 8)

In [41]:
ncboost_train

,#Chrom,Pos,Ref,Alt,NCBoost_region,NCBoost_closest_gene_name,NCBoost_Score,NCBoost_chr_rank_perc
0,16,3788684,G,C,intronic,CREBBP,0.423996,0.970256
1,X,153136500,G,A,intronic,L1CAM,0.798843,0.994768
2,11,47368616,C,T,intronic,MYBPC3,0.546437,0.989795
3,16,1576595,C,T,intronic,IFT140,0.389003,0.963604
4,2,47635062,T,G,intronic,MSH2,0.036506,0.426788
...,...,...,...,...,...,...,...,...
31649,17,4805434,G,A,UTR3,C17orf107,0.191462,0.858994
31650,12,7187798,A,T,UTR3,C1R,0.055901,0.639639
31651,1,145508647,C,T,intronic,NBPF10,0.325202,0.955519
31652,1,145518051,C,G,intronic,PEX11B,0.399849,0.972397


In [46]:
train.shape

(81140, 158)

In [42]:
train_t = train.merge(ncboost_train, on=['#Chrom', 'Pos', 'Ref', 'Alt'], how='left')

In [43]:
train_t.shape

(81293, 162)

<h1>Add DeepSEA predictions to train set</h1>

Here, the functional significance DeepSEA scores are added to the train set.

TODO: also use the DeepSEA classifiers for the variants in the train set.

See: http://deepsea.princeton.edu/help/. "We provide three classifiers trained on HGMD mutations, eQTLs and GWAS lead SNPs respectively. The first is likely appropriate for de novo mutation while the later two are likely appropriate for common variants. We also provide an unsupervised score named functional signficance score - it is not biased by any variant training set but may not have as strong performance as classifiers."

In [155]:
deepsea = deepsea.drop(['name'], axis=1)

In [156]:
deepsea.columns = ['#Chrom', 'Pos', 'Ref', 'Alt', 'DeepSEA_Functional_significance_score']

In [157]:
deepsea['#Chrom'] = deepsea['#Chrom'].apply(lambda x: x[3:])

In [158]:
deepsea

,#Chrom,Pos,Ref,Alt,DeepSEA_Functional_significance_score
71717,8,145008443,A,T,0.000010
67132,6,152702540,C,T,0.000017
43968,2,122288468,C,CTTGGGGTTGCGCTACTGTCCAATGAGCGCATAGTGAGGGCAGTAC...,0.000019
8451,10,18689957,G,A,0.000020
72434,9,35657840,C,T,0.000023
...,...,...,...,...,...
28156,16,11642677,A,T,0.716680
79279,X,46358157,T,A,0.721360
15503,12,25359328,A,T,0.737480
29252,16,68867812,A,T,0.738560


In [159]:
train = train.merge(deepsea, on=['#Chrom', 'Pos', 'Ref', 'Alt'], how='left')

In [160]:
train.shape

(81140, 154)

<h1>Add Eigen predictions to train set</h1>

In [170]:
eigen

,chr,position,ref,alt,GERP_NR,GERP_RS,PhyloPri,PhyloPla,PhyloVer,PhastPri,...,PolIISig,PolIIPval,ctcfSig,ctcfPval,cmycSig,cmycPval,Eigen-raw,Eigen-phred,Eigen-PC-raw,Eigen-PC-phred
0,1,874714,C,A,3.83,1.7100,0.591,0.938,0.326,0.008,...,0.1700,8.48,3.8276,16.00,0.1533,6.42,1.895022,27.5634,5.274241,27.0578
1,1,899926,G,A,2.74,-5.4700,0.109,-0.856,1.702,0.387,...,0.0770,4.15,0.0815,1.12,0.1704,2.66,0.562761,12.1877,2.419997,18.6624
2,1,899926,G,C,2.74,-5.4700,0.109,-0.856,1.702,0.387,...,0.0770,4.15,0.0815,1.12,0.1704,2.66,0.562761,12.1877,2.419997,18.6624
3,1,899926,G,T,2.74,-5.4700,0.109,-0.856,1.702,0.387,...,0.0770,4.15,0.0815,1.12,0.1704,2.66,0.562761,12.1877,2.419997,18.6624
4,1,978847,G,A,4.13,4.1300,0.655,2.136,2.035,0.087,...,0.0535,2.86,0.4269,3.62,0.1176,2.02,1.781010,25.2996,3.432827,21.3394
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
209059,22,51137249,T,C,4.11,0.0819,-0.282,-0.359,-2.621,0.006,...,0.0773,2.83,2.1540,12.80,0.1149,3.63,0.539487,11.9861,2.551899,19.0133
209060,22,51137249,T,G,4.11,0.0819,-0.282,-0.359,-2.621,0.006,...,0.0773,2.83,2.1540,12.80,0.1149,3.63,0.539487,11.9861,2.551899,19.0133
209061,22,51182485,A,C,4.48,0.9820,-0.404,0.143,-0.032,0.528,...,0.0282,1.40,0.0541,0.98,0.0530,2.82,0.575655,12.2964,0.804696,14.0648
209062,22,51182485,A,G,4.48,0.9820,-0.404,0.143,-0.032,0.528,...,0.0282,1.40,0.0541,0.98,0.0530,2.82,0.575655,12.2964,0.804696,14.0648


In [172]:
eigen = eigen[['chr', 'position', 'ref', 'alt', 'Eigen-raw', 'Eigen-phred', 'Eigen-PC-raw', 'Eigen-PC-phred']]

In [ ]:
eigen.columns = ['#Chrom', 'Pos', 'Ref', 'Alt', 'Eigen-raw', 'Eigen-phred', 'Eigen-PC-raw', 'Eigen-PC-phred']

In [181]:
eigen['#Chrom'] = eigen['#Chrom'].apply(lambda x: str(x))

//anaconda3/envs/py3_vp/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [182]:
eigen

,#Chrom,Pos,Ref,Alt,Eigen-raw,Eigen-phred,Eigen-PC-raw,Eigen-PC-phred
0,1,874714,C,A,1.895022,27.5634,5.274241,27.0578
1,1,899926,G,A,0.562761,12.1877,2.419997,18.6624
2,1,899926,G,C,0.562761,12.1877,2.419997,18.6624
3,1,899926,G,T,0.562761,12.1877,2.419997,18.6624
4,1,978847,G,A,1.781010,25.2996,3.432827,21.3394
...,...,...,...,...,...,...,...,...
209059,22,51137249,T,C,0.539487,11.9861,2.551899,19.0133
209060,22,51137249,T,G,0.539487,11.9861,2.551899,19.0133
209061,22,51182485,A,C,0.575655,12.2964,0.804696,14.0648
209062,22,51182485,A,G,0.575655,12.2964,0.804696,14.0648


In [186]:
train = train.merge(eigen, on=['#Chrom', 'Pos', 'Ref', 'Alt'], how='left')

In [187]:
train.shape

(81140, 158)

,#Chrom,Allergy/Immunology/Infectious,Alt,AnnoType,Audiologic/Otolaryngologic,Biochemical,CCDS,CDSpos,Cardiovascular,ConsDetail,...,verPhCons,verPhyloP,inTest,sample_weight,ReMM_score,DeepSEA_Functional_significance_score,Eigen-raw,Eigen-phred,Eigen-PC-raw,Eigen-PC-phred
0,20,False,A,Intergenic,True,False,NaN,NaN,True,downstream,...,1.000,6.368,False,1.0,NaN,0.000261,NaN,NaN,NaN,NaN
1,20,False,C,Intergenic,True,False,NaN,NaN,True,downstream,...,1.000,5.259,False,1.0,NaN,NaN,NaN,NaN,NaN,NaN
2,2,False,CC,Transcript,False,False,NaN,NaN,True,"intron,non_coding",...,1.000,3.354,False,1.0,NaN,0.003451,NaN,NaN,NaN,NaN
3,4,False,G,Intergenic,False,False,NaN,NaN,False,downstream,...,0.971,3.690,False,1.0,NaN,0.003184,NaN,NaN,NaN,NaN
4,2,False,G,Intergenic,False,False,CCDS54357.1,NaN,False,downstream,...,1.000,5.579,False,1.0,NaN,0.004027,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81135,1,False,A,Intergenic,False,False,NaN,NaN,False,downstream,...,0.000,-1.084,False,0.8,0.644,0.055760,0.249654,8.66995,0.141009,10.40580
81136,17,False,T,Intergenic,False,False,CCDS45588.1,NaN,False,downstream,...,0.000,-0.894,False,0.8,0.085,0.078554,0.114326,6.60064,1.348579,15.76650
81137,2,False,G,Intergenic,False,False,NaN,NaN,False,downstream,...,0.003,-0.012,False,0.8,0.069,0.052555,0.607428,12.55540,-0.078168,7.21020
81138,22,False,G,Intergenic,False,False,NaN,NaN,False,downstream,...,0.000,-0.170,False,0.8,0.027,0.118910,0.108753,6.51278,-0.143240,5.05581


<h1>Save train set with added columns of NCtools</h1>

In [188]:
train.to_csv('../data/train_nc_nctools-predictions.txt', sep='\t', index=False)